In [16]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

In [ ]:
%%javascript
$('#menubar').hide();

In [ ]:
from simcore_sdk import node_ports
PORTS = node_ports.ports()

In [ ]:
import plotly
import plotly.offline as offline
import plotly.figure_factory as ff
import plotly.graph_objs as go
import pandas as pd
import os

offline.init_notebook_mode(connected=True)

column_name_map = {0: {"name":"time [s]"}, 
                   1: {"name":"rate"}, 
                   2: {"name":"demand"}, 
                   3: {"name":"move/(max - min) sympathetic efferent", "scale":1/0.0008}, 
                   4: {"name":"threshold indirect parasympathetic efferent", "scale":-1.0},
                   5: {"name":"direct parasympathetic efferent", "scale":-1/0.0008}, 
                   6: {"name":"recruitment = coefficient of variation"}}
data_path = await PORTS.inputs[0].get()
outputController = pd.read_csv(data_path, sep=' ', header=None, skipinitialspace=True)
column_rename_dict = {}
for i in column_name_map:
    column_rename_dict[i] = column_name_map[i]["name"]
outputController.rename(inplace=True, columns=column_rename_dict)

trace = go.Table(
    header=dict(
        values=[outputController.columns[i] for i in column_name_map.keys()],
        #values=[column_name_map[i]["name"] for i in column_name_map.keys()],
        fill=dict(color="#C2D4FF"),
        align=["left"]
    ),
    cells=dict(
        values=[outputController[outputController.columns[i]] for i in column_name_map.keys()],
        #values=[data_frame[0:8]],
        fill=dict(color="#F5F8FF"),
        align=["left"]
    )
)
data = [trace]

offline.iplot(data, filename='data-table', config={"displayModeBar": False})

In [ ]:
# define variables
NumberOfLayers = 3
_NumberOfCells = [600, 600, 600] # cells per layer
TotalNumberOfCells = sum(_NumberOfCells)
mrows, ncols = outputController.shape
#x=outputController(:,1); %time
#y1=outputController(:,2); %rate
#y2=outputController(:,3); %demand
#y3=outputController(:,4); %move/(max - min)% sympathetic efferent
#y4=-1*outputController(:,5); %threshold % indirect parasympathetic efferent
#y5=-1*outputController(:,6); %direct parasympathetic efferent
#y6=outputController(:,7); %recruitment = coefficient of variation
#y7=outputController(:,8); %first cell
FirstCellPosition=8-1; #initialize where cell information begins

def NumberOfCells(matlab_index):
    return _NumberOfCells[matlab_index-1]

In [ ]:
import random

MAX_AMOUNT_OF_POINTS_TO_DISPLAY = 50000

def create_graph(data_frame, title, x_axis_title, colors=None, 
                 lineWidths=None, show_legend=True, names=None, annotations=None, **kwargs):
    if not names:
        names = data_frame.columns
    if not lineWidths:
        lineWidths = [2] * (data_frame.columns.size-1)
    if not colors:
        default_colors = ["black", "red", "orange","green","blue"]
        colors = [default_colors[random.randrange(0, len(default_colors))] for i in (data_frame.columns.size-1)]

    # try to do some "intelligent" slicing to reduce memory footprint when rendering
    (nrows, ncols) = data_frame.shape
    SLICING=max(1, round(nrows * ncols / MAX_AMOUNT_OF_POINTS_TO_DISPLAY))
    data = [        
        go.Scatter(
            x=data_frame.iloc[0::SLICING,0],
            y=data_frame.iloc[0::SLICING,i],
            opacity=0.8,                        
            name=names[i-1],
            marker=dict(
                color = colors[i-1]
            )
        ) for i in range(1,data_frame.columns.size)
    ]
    if annotations:
        data.extend(
            [go.Scatter(
                x=[note["text"]["x"]],
                y=[note["text"]["y"]],
                text=[note["text"]["value"]],
                mode="text",
                textfont=dict(
                    size=15,
                    color=note["text"]["color"],
                ),
                showlegend=False,
            ) for note in annotations if "text" in note]
        )

    graph_title = title
    if SLICING > 1:
        size_reduction_text = "<br><sub>(data downsampled for rendering by factor %s)<sub>" %(str(SLICING))
        graph_title = graph_title + size_reduction_text
        
    layout = go.Layout(
        title=graph_title, 
        showlegend=show_legend,
        xaxis=dict(
            title=x_axis_title
        ),        
        **kwargs
        )
    fig = go.Figure(data=data, layout=layout)
    offline.iplot(fig, config={"displayModeBar": False})
    
def create_heatmap(data_frame, title, x_axis, y_axis):
    (nrows, ncols) = data_frame.shape
    SLICING=max(1, round(nrows * ncols / MAX_AMOUNT_OF_POINTS_TO_DISPLAY))
    trace = go.Heatmap(
        z=data_frame.iloc[:,0::SLICING].as_matrix(),
        x=x_axis.iloc[0::SLICING],
        y=y_axis,
        colorscale="Greys",
    )

    data = [trace]

    graph_title = title
    if SLICING > 1:
        size_reduction_text = "<br><sub>(data downsampled for rendering by factor %s)<sub>" %(str(SLICING))
        graph_title = graph_title + size_reduction_text
        
    layout = go.Layout(
        title=graph_title, 
        showlegend=False,
        xaxis=dict(
            side="top",
        ),
        yaxis=dict(
            autorange="reversed",
        )
    )
    fig = go.Figure(data=data, layout=layout)
    offline.iplot(fig, config={"displayModeBar": False})


In [ ]:
title="Heart Rate(r)"
heart_axis_colums = [0,1,2,3,5]
heart_rate_data = outputController.filter(items=[outputController.columns[i] for i in heart_axis_colums])
heart_rate_data[heart_rate_data.columns[3]] = heart_rate_data[heart_rate_data.columns[3]].apply(lambda x: x/0.0008)
heart_rate_data[heart_rate_data.columns[4]] = heart_rate_data[heart_rate_data.columns[4]].apply(lambda x: x*-1.0/0.0008)
colors = ["magenta", "black", "red", "blue"]

dpos = 0.55
annotations = [
    {"text":{"x":2800, "y":0.05, "value":"I<sub>1<sub>", "color":"red"}},    
    {"text":{"x":6300, "y":0.05, "value":"I<sub>2<sub>", "color":"red"}},
    {"text":{"x":9800, "y":0.05, "value":"I<sub>3<sub>", "color":"red"}},
    {"text":{"x":4800, "y":dpos, "value":"D<sub>1<sub>", "color":"black"}},
    {"text":{"x":8300, "y":dpos, "value":"D<sub>2<sub>", "color":"black"}},
    {"text":{"x":11800, "y":dpos, "value":"D<sub>3<sub>", "color":"black"}},
    {"text":{"x":3800, "y":0.1, "value":"R<sub>1<sub>", "color":"blue"}},
    {"text":{"x":7300, "y":0.1, "value":"R<sub>2<sub>", "color":"blue"}},
    {"text":{"x":10800, "y":0.1, "value":"R<sub>3<sub>", "color":"blue"}},
]

line_opacity = .5
shapes = [
    {"type":"line","x0":2500,"y0":-.8,"x1":2500,"y1":.8,"opacity":line_opacity,"line":{"color":"red","width":1.5}},
    {"type":"line","x0":6000,"y0":-.8,"x1":6000,"y1":.8,"opacity":line_opacity,"line":{"color":"red","width":1.5}},
    {"type":"line","x0":9500,"y0":-.8,"x1":9500,"y1":.8,"opacity":line_opacity,"line":{"color":"red","width":1.5}},
    {"type":"line","x0":4500,"y0":-.8,"x1":4500,"y1":.8,"opacity":line_opacity,"line":{"color":"black","width":1.5}},
    {"type":"line","x0":8000,"y0":-.8,"x1":8000,"y1":.8,"opacity":line_opacity,"line":{"color":"black","width":1.5}},
    {"type":"line","x0":11500,"y0":-.8,"x1":11500,"y1":.8,"opacity":line_opacity,"line":{"color":"black","width":1.5}},
    {"type":"line","x0":3500,"y0":-.8,"x1":3500,"y1":.8,"opacity":line_opacity,"line":{"color":"blue","width":1.5}},
    {"type":"line","x0":7000,"y0":-.8,"x1":7000,"y1":.8,"opacity":line_opacity,"line":{"color":"blue","width":1.5}},
    {"type":"line","x0":10500,"y0":-.8,"x1":10500,"y1":.8,"opacity":line_opacity,"line":{"color":"blue","width":1.5}},
]

create_graph(data_frame=heart_rate_data, 
             title=title, 
             colors=colors,
             x_axis_title="\u03C4(sec)",
             annotations=annotations,
            yaxis=dict(range=[-0.4, 0.8]),
            shapes=shapes)

In [ ]:
title = "Parasympathetic Cell Activity: Cardiac Level"
FirstParasympathetic=500-1
start_index = FirstCellPosition+0*TotalNumberOfCells+FirstParasympathetic
final_index = FirstCellPosition+0*TotalNumberOfCells+NumberOfCells(1)-1+1
parasympathetic_cell_activity_columns = [0]
parasympathetic_cell_activity_columns.extend(range(start_index, final_index))
parasympathetic_cell_activity_data = outputController.filter(items=[outputController.columns[i] for i in parasympathetic_cell_activity_columns])
create_graph(data_frame=parasympathetic_cell_activity_data, 
             title=title, x_axis_title="time(sec)", show_legend=False,
            colors=["blue"]*(len(parasympathetic_cell_activity_columns)-1))

In [ ]:
title = "Cell Activity: Cardiac(r), Intrathoracic(g), Central(b)"
cell_activity_columns = [0]
# cardiac
start_index = FirstCellPosition+0*TotalNumberOfCells
final_index = FirstCellPosition+0*TotalNumberOfCells+NumberOfCells(1)-1+1
cell_activity_columns.extend(range(start_index, final_index))
colors = ["red"]*(final_index - start_index)
# intrathoracic
start_index = FirstCellPosition+0*TotalNumberOfCells+NumberOfCells(1)
final_index = FirstCellPosition+0*TotalNumberOfCells+NumberOfCells(1)+NumberOfCells(2)-1+1
cell_activity_columns.extend(range(start_index, final_index))
colors.extend(["green"]*(final_index - start_index))
# central
start_index = FirstCellPosition+0*TotalNumberOfCells+NumberOfCells(1)+NumberOfCells(2)
final_index = FirstCellPosition+0*TotalNumberOfCells+NumberOfCells(1)+NumberOfCells(2)+NumberOfCells(3)-1+1
cell_activity_columns.extend(range(start_index, final_index))
colors.extend(["blue"]*(final_index - start_index))

cell_activity_data = outputController.filter(items=[outputController.columns[i] for i in cell_activity_columns])
create_graph(data_frame=cell_activity_data, 
             title=title, x_axis_title="time(sec)", show_legend=False,
            colors=colors)

In [ ]:
title = "Heart Throughput: Cardiac(r), Intrathoracic(g), Central(b)"
heart_throughput_columns = [0]
# cardiac
start_index = FirstCellPosition+1*TotalNumberOfCells
final_index = FirstCellPosition+1*TotalNumberOfCells+NumberOfCells(1)-1+1
heart_throughput_columns.extend(range(start_index, final_index))
colors = ["red"]*(final_index - start_index)
# intrathoracic
start_index = FirstCellPosition+1*TotalNumberOfCells+NumberOfCells(1)
final_index = FirstCellPosition+1*TotalNumberOfCells+NumberOfCells(1)+NumberOfCells(2)-1+1
heart_throughput_columns.extend(range(start_index, final_index))
colors.extend(["green"]*(final_index - start_index))
# central
start_index = FirstCellPosition+1*TotalNumberOfCells+NumberOfCells(1)+NumberOfCells(2)
final_index = FirstCellPosition+1*TotalNumberOfCells+NumberOfCells(1)+NumberOfCells(2)+NumberOfCells(3)-1+1
heart_throughput_columns.extend(range(start_index, final_index))
colors.extend(["blue"]*(final_index - start_index))

data = outputController.filter(items=[outputController.columns[i] for i in heart_throughput_columns])
create_graph(data_frame=data, 
             title=title, x_axis_title="time(sec)", show_legend=False,
            colors=colors)

In [ ]:
title = "Blood Throughput: Cardiac(r), Intrathoracic(g), Central(b)"
blood_throughput_columns = [0]
# cardiac
start_index = FirstCellPosition+2*TotalNumberOfCells
final_index = FirstCellPosition+2*TotalNumberOfCells+NumberOfCells(1)-1+1
blood_throughput_columns.extend(range(start_index, final_index))
colors = ["red"]*(final_index - start_index)
# intrathoracic
start_index = FirstCellPosition+2*TotalNumberOfCells+NumberOfCells(1)
final_index = FirstCellPosition+2*TotalNumberOfCells+NumberOfCells(1)+NumberOfCells(2)-1+1
blood_throughput_columns.extend(range(start_index, final_index))
colors.extend(["green"]*(final_index - start_index))
# central
start_index = FirstCellPosition+2*TotalNumberOfCells+NumberOfCells(1)+NumberOfCells(2)
final_index = FirstCellPosition+2*TotalNumberOfCells+NumberOfCells(1)+NumberOfCells(2)+NumberOfCells(3)-1+1
blood_throughput_columns.extend(range(start_index, final_index))
colors.extend(["blue"]*(final_index - start_index))

data = outputController.filter(items=[outputController.columns[i] for i in blood_throughput_columns])
create_graph(data_frame=data, 
             title=title, x_axis_title="time(sec)", show_legend=False,
            colors=colors)

In [ ]:
#carpet plot full network data set
import math
def fix(value):
    if value > 0:
        return math.floor(value)
    return math.ceil(value)

title = "Cell Activity Level, Heart and Blood Throughput {layersnumber} Layers and {cellsperlayer} Cells per Layer'".format(layersnumber=NumberOfLayers, cellsperlayer=" ".join(map(str,_NumberOfCells)))
carpet_plot_columns = [0]
start_index = FirstCellPosition
final_index = FirstCellPosition+TotalNumberOfCells-1+1
carpet_plot_columns.extend(range(start_index, final_index))

# this does not work with plotly, too much memory involved here
f = lambda x: fix(50.0*x)
data_scaled = outputController.filter(items=[outputController.columns[i] for i in range(start_index, final_index)]).applymap(f)
#data_scaled.insert(0, "time", outputController[outputController.columns[0]])
data_scaled = data_scaled.transpose()

create_heatmap(data_frame=data_scaled, x_axis=outputController.iloc[:,0], y_axis=[i for i in range(1800)], title=title)



In [ ]:
#plot layer average neighbour importance (repeated on next figure)
title = "Relative Weight in Hierarchy"
average_neighbour_importance_columns = [0]
start_index = ncols-1-2*NumberOfLayers-NumberOfLayers-1
final_index = ncols-1-2*NumberOfLayers-1
average_neighbour_importance_columns.extend(range(start_index, final_index))

colors = ["rgb({r}, {g}, {b})".format(r=255.0*(1 - i/NumberOfLayers), g=255.0*abs(.5-i/NumberOfLayers), b=255.0*abs(.25-i/NumberOfLayers)) for i in range(1, final_index-start_index + 1)]
lineWidth = [2 + 2 * i/NumberOfLayers for i in range(1, final_index-start_index + 1)]

names = ['Cardiac','Intrathoracic','Central']
data = outputController.filter(items=[outputController.columns[i] for i in average_neighbour_importance_columns])
data.iloc[:,1:] = data.iloc[:,1:].div(outputController.iloc[:,ncols-1], axis=0)


ipos = 0.32
rpos = ipos
annotations = [
    {"text":{"x":2800, "y":ipos, "value":"I<sub>1<sub>", "color":"red"}},    
    {"text":{"x":6300, "y":ipos, "value":"I<sub>2<sub>", "color":"red"}},
    {"text":{"x":9800, "y":ipos, "value":"I<sub>3<sub>", "color":"red"}},
    {"text":{"x":4800, "y":.35, "value":"D<sub>1<sub>", "color":"black"}},
    {"text":{"x":8300, "y":.35, "value":"D<sub>2<sub>", "color":"black"}},
    {"text":{"x":11800, "y":.35, "value":"D<sub>3<sub>", "color":"black"}},
    {"text":{"x":3800, "y":rpos, "value":"R<sub>1<sub>", "color":"blue"}},
    {"text":{"x":7300, "y":rpos, "value":"R<sub>2<sub>", "color":"blue"}},
    {"text":{"x":10800, "y":rpos, "value":"R<sub>3<sub>", "color":"blue"}},
]

line_opacity = .5
shapes = [
    {"type":"line","x0":2500,"y0":0,"x1":2500,"y1":.8,"opacity":line_opacity,"line":{"color":"red","width":1.5}},
    {"type":"line","x0":6000,"y0":0,"x1":6000,"y1":.8,"opacity":line_opacity,"line":{"color":"red","width":1.5}},
    {"type":"line","x0":9500,"y0":0,"x1":9500,"y1":.8,"opacity":line_opacity,"line":{"color":"red","width":1.5}},
    {"type":"line","x0":4500,"y0":0,"x1":4500,"y1":.8,"opacity":line_opacity,"line":{"color":"black","width":1.5}},
    {"type":"line","x0":8000,"y0":0,"x1":8000,"y1":.8,"opacity":line_opacity,"line":{"color":"black","width":1.5}},
    {"type":"line","x0":11500,"y0":0,"x1":11500,"y1":.8,"opacity":line_opacity,"line":{"color":"black","width":1.5}},
    {"type":"line","x0":3500,"y0":0,"x1":3500,"y1":.8,"opacity":line_opacity,"line":{"color":"blue","width":1.5}},
    {"type":"line","x0":7000,"y0":0,"x1":7000,"y1":.8,"opacity":line_opacity,"line":{"color":"blue","width":1.5}},
    {"type":"line","x0":10500,"y0":0,"x1":10500,"y1":.8,"opacity":line_opacity,"line":{"color":"blue","width":1.5}},
]

create_graph(data_frame=data, 
             title=title, x_axis_title="time(sec)", show_legend=True,
            colors=colors,
            lineWidths=lineWidth,
            names=names,
            annotations=annotations,
            yaxis=dict(range=[0,0.4]),
            shapes=shapes)

In [ ]:
# plot layer neighbour importance standard deviation (shifted by +1 from
# previous graph which plots the layer neighbour importance average
title = "Layer: Neighbour Importance Standard Deviation"
average_neighbour_importance_std_columns = [0]
start_index = ncols-1-2*NumberOfLayers+1-1
final_index = ncols-1-1+1
average_neighbour_importance_std_columns.extend(range(start_index, final_index, 2))

colors = ["rgb({r}, {g}, {b})".format(r=255.0*(1 - i/2.0/NumberOfLayers), g=255.0*(1 - i/2.0/NumberOfLayers), b=255.0*(1 - i/2.0/NumberOfLayers)) for i in range(1, final_index-start_index, 2)]
lineWidth = [1 + 2 * i/NumberOfLayers for i in range(1, final_index-start_index, 2)]
names = ['Cardiac','Intrathoracic','Central']
data = outputController.filter(items=[outputController.columns[i] for i in average_neighbour_importance_std_columns])
create_graph(data_frame=data, 
             title=title, x_axis_title="time(sec)", show_legend=True,
            colors=colors,
            lineWidths=lineWidth,
            names=names)